<a href="https://colab.research.google.com/github/sassgabai/pyspark-projects/blob/main/web_scrapping_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install bs4

In [31]:
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession

In [85]:
spark = SparkSession.builder.getOrCreate()

In [32]:
URL = 'https://www.amazon.com/s?k=dragon+ball+z&language=en_US&crid=221T1AZ8FNVV9&sprefix=dragon+ball+%2Caps%2C235&ref=nb_sb_noss_2'

In [33]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [34]:
data = requests.get(URL, headers=HEADERS)

In [35]:
soup = BeautifulSoup(data.content, 'html.parser')

In [38]:
get_links = soup.find_all('a', attrs={'class':'a-link-normal s-no-outline'})

In [ ]:
links_list =[]

for link in get_links:
  link_url = f'https://amazon.com{link.get("href")}'
  links_list.append(link_url)

In [99]:
data_schema = ['description','price','title']
data_collected = []

In [100]:
for link in links_list[:5]:
  link_data = requests.get(link, headers=HEADERS)
  link_soup = BeautifulSoup(link_data.content, 'html.parser')

  get_header = link_soup.find('span', attrs={'class':'a-size-large product-title-word-break'}).text.strip()
  get_price = link_soup.find('div', attrs={'class':'a-section a-spacing-micro'})\
                       .find('span', attrs={'class':'a-offscreen'}).text.strip().replace('$','')


  get_description = link_soup.find('div', attrs={'class':'a-section a-spacing-medium a-spacing-top-small'})\
                             .find('ul', attrs={'class':'a-unordered-list a-vertical a-spacing-mini'})\

  for desc in get_description:
    list_desc = [desc.text.strip() for desc in get_description if desc.text.strip()]

  data_collected.append({
      'title': get_header,
      'price': get_price,
      'description': list_desc
  })



In [101]:
df = spark.createDataFrame(data_collected, data_schema)\
          .select('title','description','price')

In [ ]:
df.show(100, truncate=False)